In [1]:
little_star="tinynotation: 4/4 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2 g4 g4 f4 f4 e4 e4 d2 g4 g4 f4 f4 e4 e4 d2 c4 c4 g4 g4 a4 a4 g2 f4 f4 e4 e4 d4 d4 c2" 
spring_picnic="tinynotation: 4/8 g8 e8 g8 e8 g8 a8 g4 e8 g8 e8 c8 d8 e8 c4 g8 e8 g8 e8 g8 a8 g4 b8 a8 g8 e8 d8 e8 c4" 
butterfly="tinynotation: 2/4 g8 e8 e4 f8 d8 d4 c8 d8 e8 f8 g8 g8 g4 g8 e8 e8 e8 f8 d8 d4 c8 e8 g8 g8 e8 e8 e4 d8 d8 d8 d8 d8 e8 f4 e8 e8 e8 e8 e8 f8 g4 g8 e8 e4 f8 d8 d4 c8 e8 g8 g8 e8 e8 e4"

In [ ]:
little_star
spring_picnic
butterfly

In [2]:
import music21
import numpy as np

C:\Users\user\miniconda3\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
note2num = {'c':1,'d':2,'e':3,'f':4,'g':5,'a':6,'b':7}
num2note = {}
for key,value in  note2num.items():
    num2note[value] = key  

In [4]:
# 시계열 데이터를 학습용과 정답을 생성
def makeTrainDataAndClass(seq,window):
    X,Y = [],[]
    for i in range(len(seq) - window):
        x = seq[i:window+i]
        y = seq[i+window]
        X.append(x); Y.append(y)
    return np.array(X), np.array(Y)     

In [5]:
musicData = little_star.split()[2:] + spring_picnic.split()[2:] + butterfly.split()[2:]

In [18]:
# ABC 악보 ->데이터  c4
# note2num 딕셔너리 구조를 적용해서 c4처럼 문자열의 앞 문자만 추출해서 해당 문자를 키로 입력
note2num['c4'[0]]
abcToData = []
for abc in musicData:
    abcToData.append([ note2num[abc[0]], int(abc[1]) ])
# abcToData    
# eval  함수나 수식을 문자열로 표했을때.->원래대로 풀어쓰기 위한 용도

In [20]:
# 위의 코드를 리스트컴프리핸선으로 변환
abcToData = [ [ note2num[abc[0]], int(abc[1]) ]  for abc in musicData ]

In [22]:
x_train,y_train =  makeTrainDataAndClass(abcToData,7)

In [23]:
x_train.shape, y_train.shape

((117, 7, 2), (117, 2))

In [9]:
# LSTM 모델 생성
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense

In [27]:
# 모델 설계
model =Sequential()
model.add(LSTM(256,activation='relu',input_shape =x_train.shape[1:] ))
model.add(Dense(156,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(2))
model.compile(loss='mse',optimizer='rmsprop',metrics=['acc','mae'])

In [28]:
earlyStop = tf.keras.callbacks.EarlyStopping(patience=5) # val_loss

In [30]:
hist = model.fit(x_train,y_train,epochs=200,batch_size=1,validation_data=(x_train,y_train),
                callbacks=earlyStop)

Epoch 1/200
117/117 [==============================] - 4s 34ms/step - loss: 3.1500 - acc: 0.7949 - mae: 1.4176 - val_loss: 6.5823 - val_acc: 0.7949 - val_mae: 2.1716
Epoch 2/200
117/117 [==============================] - 3s 27ms/step - loss: 2.7719 - acc: 0.7949 - mae: 1.3004 - val_loss: 4.9531 - val_acc: 0.7949 - val_mae: 1.8610
Epoch 3/200
117/117 [==============================] - 3s 25ms/step - loss: 2.7813 - acc: 0.8034 - mae: 1.2960 - val_loss: 1.9933 - val_acc: 0.7949 - val_mae: 1.0520
Epoch 4/200
117/117 [==============================] - 3s 24ms/step - loss: 2.6616 - acc: 0.7863 - mae: 1.2911 - val_loss: 1.8696 - val_acc: 0.7949 - val_mae: 1.0203
Epoch 5/200
117/117 [==============================] - 3s 29ms/step - loss: 2.4964 - acc: 0.7949 - mae: 1.2213 - val_loss: 3.0717 - val_acc: 0.7949 - val_mae: 1.4787
Epoch 6/200
117/117 [==============================] - 4s 31ms/step - loss: 2.4762 - acc: 0.7863 - mae: 1.2320 - val_loss: 2.0254 - val_acc: 0.7949 - val_mae: 1.0483
Epoc

In [31]:
# 학습이 끝났고...
# 예측 - 곡을 예측
# 랜덤하게 시작과 종료를 선택해서 기존 x_train에서 분할하고
# 분할한 데이터를 예측해서  -ABC 만든다음 이 과정을 반복한다.

# 1. 랜덤하게 두개 숫자 생성, 셈플데이터 추출
# 2. 모델에 예측
# 3. ABC 악보로 변경
# 4. 1 2 3반복

In [110]:
#1. 
def makeMusic1():
    temp = np.sort(sample(range(len(x_train)),2))
    return x_train[ temp[0]:temp[1] ]

In [111]:
#2 
def makeMusic2():
    x_pre_predict = makeMusic1()
    x_predict =  np.round(model.predict(x_pre_predict)).astype('uint8')
    return x_predict

In [114]:
# 3 데이터 ->ABC 악보
def makeMusic3():
    x_predict = makeMusic2();
    dataToAbc = []
    for nums in x_predict:
        dataToAbc.append(num2note[nums[0]]+str(nums[1]))
    return ' '.join(dataToAbc)

In [127]:
new_music = 'tinynotation: 4/8 ' + makeMusic3() +' ' + makeMusic3()+' ' + makeMusic3()
new_music

1/1 [==============================] - 0s 25ms/step


'tinynotation: 4/8 e8 e6 e7 f7 e7 f7 g6 f9 e9 e8 e8 e8 e9 e8 e8 d6 e8 e9 e8 e9 f8 e8 e6 e8 e9 e8 e9 e8 e8 e6 f9 e8 e8 e8 e8 e8 e6'

In [128]:
music21.converter.parse(new_music).show('mid')

In [151]:
############  데이터를 수평이 아닌 수직으로 병합해서 데이터를 생성
print(len(little_star.split()))
print(len(spring_picnic.split()))
print(len(butterfly.split()))

44
30
56


In [162]:
#  mid음악 3개의 길이를 맞추기 위해  가장적은걸 기준으로 자른다.
MAX_NUM = 30
musicdata1 = little_star.split()[2:MAX_NUM] 
musicdata2 = spring_picnic.split()[2:MAX_NUM] 
musicdata3 = butterfly.split()[2:MAX_NUM] 

In [163]:
# 길이를 확인
len(musicdata1), len(musicdata2), len(musicdata3)

(28, 28, 28)

In [164]:
# 문자형 데이터를 수치형 데이터로 변형
musicdata1 =  np.array( [ [note2num[i[0]],int(i[1])] for i in musicdata1])
musicdata2 =  np.array( [ [note2num[i[0]],int(i[1])] for i in musicdata2])
musicdata3 =  np.array( [ [note2num[i[0]],int(i[1])] for i in musicdata3])

In [166]:
print(musicdata1.shape)
musicdata1[:6]

(28, 2)


array([[1, 4],
       [1, 4],
       [5, 4],
       [5, 4],
       [6, 4],
       [6, 4]])

In [184]:
# vmergeData = np.vstack( [musicdata1,musicdata2,musicdata3] )
# vmergeData.shape
vmergeData = np.array([musicdata1,musicdata2,musicdata3])
vmergeData.shape

(3, 28, 2)

In [185]:
# 시계열 데이터를 학습용과 정답을 생성
# 3차원일 경우
def makeTrain2DimDataAndClass(seq,window):
    X,Y = [],[]
    assert len(seq.shape) == 3
    for i in range(seq.shape[1] - window):
        x = seq[:,i:window+i]
        y = seq[:,i+window]
        X.append(x); Y.append(y)
    return np.array(X), np.array(Y)  

In [186]:
# LSTM에 들어갈 시계열 형태의 데이터 생성
x_train,y_train = makeTrain2DimDataAndClass(vmergeData,4)
x_train.shape,y_train.shape

((24, 3, 4, 2), (24, 3, 2))

In [189]:
x_train = x_train.reshape(24,3,-1)
y_train = y_train.reshape(24,-1)
x_train.shape,y_train.shape

((24, 3, 8), (24, 6))

In [191]:
# 모델
model=Sequential()
model.add(LSTM(128,activation='relu', input_shape = x_train.shape[1:]))
model.add(Dense(6))
model.compile(loss='mse',optimizer='adam',metrics=['mae','acc'])
model.fit(x_train,y_train,epochs=200,batch_size=1)

Epoch 1/200
24/24 [==============================] - 1s 11ms/step - loss: 7.6027 - mae: 2.1071 - acc: 0.2500
Epoch 2/200
24/24 [==============================] - 0s 10ms/step - loss: 2.6029 - mae: 1.3001 - acc: 0.3750
Epoch 3/200
24/24 [==============================] - 0s 9ms/step - loss: 2.7317 - mae: 1.3369 - acc: 0.7083
Epoch 4/200
24/24 [==============================] - 0s 11ms/step - loss: 2.2030 - mae: 1.1273 - acc: 0.1667
Epoch 5/200
24/24 [==============================] - 0s 11ms/step - loss: 2.0751 - mae: 1.1486 - acc: 0.8750
Epoch 6/200
24/24 [==============================] - 0s 11ms/step - loss: 2.1536 - mae: 1.1763 - acc: 0.5000
Epoch 7/200
24/24 [==============================] - 0s 11ms/step - loss: 1.9792 - mae: 1.0594 - acc: 0.6250
Epoch 8/200
24/24 [==============================] - 0s 11ms/step - loss: 1.9820 - mae: 1.0922 - acc: 0.4583
Epoch 9/200
24/24 [==============================] - 0s 11ms/step - loss: 1.7768 - mae: 1.0654 - acc: 1.0000
Epoch 10/200
24/24 [

24/24 [==============================] - 0s 11ms/step - loss: 0.0870 - mae: 0.1933 - acc: 0.7083
Epoch 75/200
24/24 [==============================] - 0s 11ms/step - loss: 0.1019 - mae: 0.2225 - acc: 0.8750
Epoch 76/200
24/24 [==============================] - 0s 11ms/step - loss: 0.0859 - mae: 0.1890 - acc: 0.5000
Epoch 77/200
24/24 [==============================] - 0s 14ms/step - loss: 0.0626 - mae: 0.1728 - acc: 0.8750
Epoch 78/200
24/24 [==============================] - 0s 13ms/step - loss: 0.0812 - mae: 0.1863 - acc: 0.7500
Epoch 79/200
24/24 [==============================] - 0s 12ms/step - loss: 0.0825 - mae: 0.1972 - acc: 0.6250
Epoch 80/200
24/24 [==============================] - 0s 11ms/step - loss: 0.0504 - mae: 0.1443 - acc: 0.7083
Epoch 81/200
24/24 [==============================] - 0s 12ms/step - loss: 0.0526 - mae: 0.1562 - acc: 0.7500
Epoch 82/200
24/24 [==============================] - 0s 14ms/step - loss: 0.0571 - mae: 0.1724 - acc: 0.7083
Epoch 83/200
24/24 [===

24/24 [==============================] - 0s 12ms/step - loss: 0.0183 - mae: 0.1014 - acc: 0.6250
Epoch 149/200
24/24 [==============================] - 0s 10ms/step - loss: 0.0270 - mae: 0.1226 - acc: 0.7083
Epoch 150/200
24/24 [==============================] - 0s 11ms/step - loss: 0.0366 - mae: 0.1498 - acc: 0.6250
Epoch 151/200
24/24 [==============================] - 0s 13ms/step - loss: 0.0403 - mae: 0.1551 - acc: 0.6250
Epoch 152/200
24/24 [==============================] - 0s 11ms/step - loss: 0.0333 - mae: 0.1495 - acc: 0.7917
Epoch 153/200
24/24 [==============================] - 0s 10ms/step - loss: 0.0557 - mae: 0.1903 - acc: 0.5833
Epoch 154/200
24/24 [==============================] - 0s 19ms/step - loss: 0.0370 - mae: 0.1445 - acc: 0.7500
Epoch 155/200
24/24 [==============================] - 0s 9ms/step - loss: 0.0439 - mae: 0.1571 - acc: 0.6667
Epoch 156/200
24/24 [==============================] - 0s 9ms/step - loss: 0.0396 - mae: 0.1548 - acc: 0.5417
Epoch 157/200
24/

In [211]:
# 첫소절 5
predict = model.predict(x_train[:5])
predict = predict.astype('uint8')

# song1 = 'tinynotation: 4/4 '
# song2 = 'tinynotation: 4/8 '
# song3 = 'tinynotation: 2/4 '
song1 = ''
for i in range(5):
    song1 += ' '.join(num2note[round(predict[i][0])]) + str(round(predict[i][1])) + ' '
    song1 += ' '.join(num2note[round(predict[i][2])]) + str(round(predict[i][3])) + ' '
    song1 += ' '.join(num2note[round(predict[i][4])]) + str(round(predict[i][5])) + ' '
    

1/1 [==============================] - 0s 20ms/step


In [213]:
song1 = 'tinynotation: 4/8 ' + song1
song1

'tinynotation: 4/8 g3 g8 c7 g3 g7 c3 f2 f3 c7 e4 d7 d8 e3 f8 e7 '